# SIMLAB2


### Control de errores 
Deben comprobarse los errores de TODAS las llamadas a sistema (excepto el write por pantalla), controlar los argumentos de entrada y definir la función **Usage()**, y, además, realizarse los controles adecuados según pida el enunciado.

### Parte 1 
Escribe un programa en C, llámalo **userprocs.c** que mediante las herramientas vistas en clase genere un resultado basado en el de la siguiente linea de comandos:

$> ps -eaf | grep username | wc -l

El resultado de userprocs por pantalla debe ser, únicamente, una frase tal como:

“El usuario username está ejecutando XX procesos”


**NOTA:** las comunicaciones entre ps, grep y wc deben hacerse con pipes ordinarias.
Cualquier otra comunicación entre procesos se debe hacer mediante pipes con nombre.

**EJEMPLO**

$> ps -eaf | grep alumne | wc -l

53

$> ./userprocs alumne

El usuario alumne está ejecutando 53 procesos

$>

#### Programación

In [ ]:
%%file userprocs.c
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <errno.h>
#include <string.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <signal.h>

void error_y_exit(char *str)
{
	perror(str);
	exit(EXIT_FAILURE);
}

void Usage()
{
	char buffer[256];
	sprintf(buffer, "USAGE: userprocs username\n");
	write(1, buffer, strlen(buffer));
	exit(EXIT_FAILURE);
}

int main(int argc, char *argv[])
{

	int ps2grep[2], grep2wc[2];
	int read_wc2parent, write_wc2parent;
	int ret;
	char wcbuffer[256];
	char buffer[500];

	if (argc != 2)
		Usage();

	pipe(ps2grep);
	pipe(grep2wc);

	ret = mknod("wc2parent.fifo", S_IFIFO | 0666, 0);	// Creo la NAMED-PIPE, control de EEXIST
	if ((ret == -1) && (errno != EEXIST))
		error_y_exit("Creando la named pipe");

	switch (fork())		//fork para mutar a ps
	{
	case -1:
		error_y_exit("Fork para ps");
		break;
	case 0:
		dup2(ps2grep[1], 1);
		close(ps2grep[1]);	//redirecció Salida Estandar a pipe. Cierre canal duplicado.

		close(ps2grep[0]);	//cierro canales de pipe que no uso
		close(grep2wc[0]);	//evitando bloqueos
		close(grep2wc[1]);

//kill(getpid(),SIGSTOP);

		execlp("ps", "ps", "-eaf", (char *)NULL);
		error_y_exit("Mutación a ps");
		break;

	}

	switch (fork())		//fork para mutar a grep
	{
	case -1:
		error_y_exit("Fork para grep");
		break;
	case 0:
		dup2(ps2grep[0], 0);
		close(ps2grep[0]);	//redirecció entrada Estandar a pipe. Cierre canal duplicado.

		dup2(grep2wc[1], 1);
		close(grep2wc[1]);	//redirecció Salida Estandar a pipe. Cierre canal duplicado.

		close(ps2grep[1]);	//cierro canales de pipe que no uso
		close(grep2wc[0]);	//evitando bloqueos

//kill(getpid(),SIGSTOP);

		execlp("grep", "grep", argv[1], (char *)NULL);
		error_y_exit("Mutación a grep");
		break;

	}

	switch (fork())		//fork para mutar a wc
	{
	case -1:
		error_y_exit("Fork para wc");
		break;
	case 0:
		dup2(grep2wc[0], 0);
		close(grep2wc[0]);	//redirecció entrada Estandar a pipe. Cierre canal duplicado.

		write_wc2parent = open("wc2parent.fifo", O_WRONLY, 0);
		dup2(write_wc2parent, 1);
		close(write_wc2parent);	//redirecció Salida Estandar a named pipe. Cierre canal duplicado. Bloqueo de open

		close(ps2grep[0]);
		close(ps2grep[1]);	//cierro canales de pipe que no uso
		close(grep2wc[1]);	//evitando bloqueos

//kill(getpid(),SIGSTOP);

		execlp("wc", "wc", "-l", (char *)NULL);
		error_y_exit("Mutación a wc");
		break;

	}

///PADRE
//Cierro los canales de pipe que no uso
	close(grep2wc[0]);
	close(grep2wc[1]);
	close(ps2grep[0]);
	close(ps2grep[1]);

	read_wc2parent = open("wc2parent.fifo", O_RDONLY, 0);	//abre la named-pipe. Hay bloqueo

//kill(getpid(),SIGSTOP);
	int valor;
	while ((ret = read(read_wc2parent, wcbuffer, 256)) > 0)	//la lectura se bloquea hasta que wc acaba
	{
		valor = atoi(wcbuffer);	//nos guardamos el valor para escribirlo en el fichero
		wcbuffer[ret - 1] = '\0';
		sprintf(buffer,
			"El usuario %s está ejecutando %s procesos\n", argv[1],
			wcbuffer);
		write(1, buffer, strlen(buffer));
	}

	close(read_wc2parent);
	while (waitpid(-1, NULL, 0) > 0) ;
}




#### Compilación

In [ ]:
%%bash
gcc -o userprocs.exe userprocs.c

#### Ejecución

In [ ]:
%%bash
./userprocs.exe root

### Parte 2 
Además, el resultado de la ejecución (el valor resultado de wc, en el ejemplo 53) se debe guardar en un fichero con nombre **“username.dat”**. El valor, en formato binario (entero), debe escribirse siempre al final de fichero.

Si el fichero no existe, debe crearse con permisos de lectura/escritura para el usuario creador, y con permisos de solo lectura para el grupo de usuario y el resto.
Si el fichero existe, se debe preguntar al usuario si desea sobreescribirlo o añadir los datos al final.

In [ ]:
%%file userprocs_p2.c
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <errno.h>
#include <string.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <signal.h>

void error_y_exit(char *str)
{
	perror(str);
	exit(EXIT_FAILURE);
}

void Usage()
{
	char buffer[256];
	sprintf(buffer, "USAGE: userprocs username\n");
	write(1, buffer, strlen(buffer));
	exit(EXIT_FAILURE);
}

int main(int argc, char *argv[])
{

	int ps2grep[2], grep2wc[2];
	int read_wc2parent, write_wc2parent;
	int ret;
	char wcbuffer[256];
	char buffer[500];

	if (argc != 2)
		Usage();

	pipe(ps2grep);
	pipe(grep2wc);

	ret = mknod("wc2parent.fifo", S_IFIFO | 0666, 0);	// Creo la NAMED-PIPE, control de EEXIST
	if ((ret == -1) && (errno != EEXIST))
		error_y_exit("Creando la named pipe");

	switch (fork())		//fork para mutar a ps
	{
	case -1:
		error_y_exit("Fork para ps");
		break;
	case 0:
		dup2(ps2grep[1], 1);
		close(ps2grep[1]);	//redirecció Salida Estandar a pipe. Cierre canal duplicado.

		close(ps2grep[0]);	//cierro canales de pipe que no uso
		close(grep2wc[0]);	//evitando bloqueos
		close(grep2wc[1]);

//kill(getpid(),SIGSTOP);

		execlp("ps", "ps", "-eaf", (char *)NULL);
		error_y_exit("Mutación a ps");
		break;

	}

	switch (fork())		//fork para mutar a grep
	{
	case -1:
		error_y_exit("Fork para grep");
		break;
	case 0:
		dup2(ps2grep[0], 0);
		close(ps2grep[0]);	//redirecció entrada Estandar a pipe. Cierre canal duplicado.

		dup2(grep2wc[1], 1);
		close(grep2wc[1]);	//redirecció Salida Estandar a pipe. Cierre canal duplicado.

		close(ps2grep[1]);	//cierro canales de pipe que no uso
		close(grep2wc[0]);	//evitando bloqueos

//kill(getpid(),SIGSTOP);

		execlp("grep", "grep", argv[1], (char *)NULL);
		error_y_exit("Mutación a grep");
		break;

	}

	switch (fork())		//fork para mutar a wc
	{
	case -1:
		error_y_exit("Fork para wc");
		break;
	case 0:
		dup2(grep2wc[0], 0);
		close(grep2wc[0]);	//redirecció entrada Estandar a pipe. Cierre canal duplicado.

		write_wc2parent = open("wc2parent.fifo", O_WRONLY, 0);
		dup2(write_wc2parent, 1);
		close(write_wc2parent);	//redirecció Salida Estandar a named pipe. Cierre canal duplicado. Bloqueo de open

		close(ps2grep[0]);
		close(ps2grep[1]);	//cierro canales de pipe que no uso
		close(grep2wc[1]);	//evitando bloqueos

//kill(getpid(),SIGSTOP);

		execlp("wc", "wc", "-l", (char *)NULL);
		error_y_exit("Mutación a wc");
		break;

	}

///PADRE
//Cierro los canales de pipe que no uso
	close(grep2wc[0]);
	close(grep2wc[1]);
	close(ps2grep[0]);
	close(ps2grep[1]);

	read_wc2parent = open("wc2parent.fifo", O_RDONLY, 0);	//abre la named-pipe. Hay bloqueo

//kill(getpid(),SIGSTOP);
	int valor;
	while ((ret = read(read_wc2parent, wcbuffer, 256)) > 0)	//la lectura se bloquea hasta que wc acaba
	{
		valor = atoi(wcbuffer);	//nos guardamos el valor para escribirlo en el fichero
		wcbuffer[ret - 1] = '\0';
		sprintf(buffer,
			"El usuario %s está ejecutando %s procesos\n", argv[1],
			wcbuffer);
		write(1, buffer, strlen(buffer));
	}

	close(read_wc2parent);
	while (waitpid(-1, NULL, 0) > 0) ;

//apertura/creación del fichero
	char nfich[256], pregunta[1000], respuesta[80];
	sprintf(nfich, "%s.dat", argv[1]);	//nombre del fichero
	int fichero = open(nfich, O_WRONLY | O_CREAT | O_EXCL, 0644);
	if (fichero == -1) {
		if (errno != EEXIST)
			error_y_exit("crear fichero");
		else {
			ret =
			    sprintf(pregunta,
				    "\n!!!! El fichero ya existe: Pulsa S + return para SOBREESCRIBIRLO, \ncualquier otra cosa para + return para AÑADIR los datos al final:  ");
			write(1, pregunta, ret);
			read(0, respuesta, 80);
			if (strcmp(respuesta, "S\n") == 0) {
				fichero =
				    open(nfich, O_WRONLY | O_CREAT | O_TRUNC,
					 0644);
			} else {
				fichero = open(nfich, O_WRONLY, 0644);
			}
		}
	}
	lseek(fichero, 0, SEEK_END);
	write(fichero, &valor, sizeof(int));
	close(fichero);
}

#### Compilación

In [ ]:
%%bash
gcc -o userprocs_p2.exe userprocs_p2.c

#### Ejecución

In [ ]:
%%bash
printf "S\n" | ./userprocs_p2.exe root 

#### Prueba

con xxd en formato hexadecimal o más abajo con el programa de leer enteros

In [ ]:
%%bash
xxd root.dat

## Leer enteros

Programa que lee enteros en formato interno de máquina por la entrada estandar y los escribe en formato "humano" por la salida estandard

#### Código

In [ ]:
%%file lee_ints.c
#include <unistd.h>
#include <stdio.h>
void
main ()
{
  int leido, ret, spr;
  char buff[64];
  while ((ret = read (0, &leido, sizeof (int))) > 0)
    {
      spr = sprintf (buff, "%d\n", leido);
      write (1, buff, spr);
    }
}

#### Compilación lee_ints 

In [ ]:
%%bash
gcc -o lee_ints lee_ints.c

#### Prueba

In [ ]:
%%bash
./lee_ints < millanva.dat

## CLEAN

In [ ]:
%%bash
rm *.exe
rm *.dat